# Benchmark 1

We study the Hamiltonian of the Heisenberg model with periodic boundary conditions.

In [1]:
using CondensedMatterSOS
@spin σ[1:3]
heisenberg_hamiltonian(σ, true)

# Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

using CSDP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => false,
)

MathOptInterface.OptimizerWithAttributes(Main.##258.var"#1#2"(), Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.Silent() => false])

We can compute a lower bound `-2√2` to the ground state energy as follow:

In [2]:
include(joinpath(dirname(dirname(pathof(CondensedMatterSOS))), "examples", "symmetry.jl"))
function hamiltonian_energy(N, maxdegree, solver; symmetry=true, consecutive=false, kws...)
    @spin σ[1:N]
    function action(mono::CondensedMatterSOS.SpinMonomial, el::DirectSum)
        isempty(mono.variables) && return 1 * mono
        sign = 1
        vars = map(values(mono.variables)) do var
            rel_id = var.id - σ[1][1].id
            rel_index = var.index + 1
            @assert σ[rel_index][rel_id + 1] == var
            id = ((rel_id + el.c.id) % el.c.n) + σ[1][1].id
            index = (rel_index^el.k.p) - 1
            new_var = CondensedMatterSOS.SpinVariable(id, index)
            if el.k.k.id != 0 && el.k.k.id != index + 1
                sign *= -1
            end
            return new_var
        end
        return sign * CondensedMatterSOS.SpinMonomial(vars)
    end
    function action(term::CondensedMatterSOS.SpinTerm, el::DirectSum)
        return MP.coefficient(term) * action(MP.monomial(term), el)
    end
    function action(poly::CondensedMatterSOS.SpinPolynomial, el::DirectSum)
        return MP.polynomial([action(term, el) for term in MP.terms(poly)])
    end
    H = heisenberg_hamiltonian(σ, true)
    G = Lattice1Group(N)
    cone = NonnegPolyInnerCone{SumOfSquares.COI.HermitianPositiveSemidefiniteConeTriangle}()
    @assert iseven(maxdegree)
    cert = SumOfSquares.Certificate.FixedBasis(
        cone,
        MonomialBasis(MP.monomials(σ[1], 0:div(maxdegree, 2), consecutive=consecutive)),
    )
    display(cert.basis.monomials)
    certificate = SymmetricIdeal(
        cert,
        G,
        action,
    )
    if symmetry
        energy(H, maxdegree, solver; certificate = certificate, kws...)
    else
        energy(H, maxdegree, solver; kws...)
    end
end
bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
    symmetry = false,
    sparsity = NoSparsity(),
)
bound

Iter: 11 Ap: 9.58e-01 Pobj: -1.8000000e+01 Ad: 9.58e-01 Dobj: -1.8000000e+01 
Success: SDP solved
Primal objective value: -1.8000000e+01 
Dual objective value: -1.8000000e+01 
Relative primal infeasibility: 1.96e-14 
Relative dual infeasibility: 1.59e-09 
Real Relative Gap: 2.41e-09 
XZ Relative Gap: 3.09e-09 
DIMACS error measures: 5.14e-14 0.00e+00 5.43e-09 0.00e+00 2.41e-09 3.09e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6378771e+01 Ad: 4.06e-01 Dobj: -4.2861712e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5224975e+01 Ad: 9.54e-01 Dobj: -7.6357706e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.5310322e+01 Ad: 8.85e-01 Dobj: -1.2848520e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -8.3981756e+00 Ad: 7.63e-01 Dobj: -3.0908816e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -6.6372507e+00 Ad: 8.87e-01 Dobj: -5.5100925e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0459643e+00 Ad: 9.38e-01 Dobj: -5.9416047e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -6.0024942e+00 Ad: 9.52e-0

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

-6.000000023230198

We can see that the moment matrix uses all monomials:

In [3]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

# Symmetry reduction

We can reduce the computation using symmetry reduction as follows.

In [4]:
using CondensedMatterSOS

bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
)
bound

Iter: 11 Ap: 9.60e-01 Pobj: -6.0000000e+00 Ad: 9.60e-01 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 1.41e-16 
Relative dual infeasibility: 2.23e-09 
Real Relative Gap: 1.82e-09 
XZ Relative Gap: 4.09e-09 
DIMACS error measures: 2.09e-16 0.00e+00 3.46e-09 0.00e+00 1.82e-09 4.09e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.42e-01 Pobj: -1.0028738e+01 Ad: 7.52e-01 Dobj: -1.5475638e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -1.9863596e+01 Ad: 9.31e-01 Dobj: -3.1176016e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -9.3630857e+00 Ad: 9.02e-01 Dobj: -4.7171806e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -6.3308914e+00 Ad: 9.06e-01 Dobj: -5.7530200e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -6.0172805e+00 Ad: 9.36e-01 Dobj: -5.9769435e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0008220e+00 Ad: 9.59e-01 Dobj: -5.9986671e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -6.0000533e+00 Ad: 9.87e-0

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

-6.00000000578403

The reduction is obtained by block diagonalizing with a change of polynomial
basis to the isotypical basis.

In [5]:
display([M.basis.polynomials for M in ν.sub_moment_matrices])

[M.basis.polynomials for M in ν.sub_moment_matrices]

7-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [σˣ₁ + σˣ₂]
 [σʸ₁ + σʸ₂]
 [σᶻ₁ + σᶻ₂]
 [σˣ₁ + -σˣ₂]
 [σʸ₁ + -σʸ₂]
 [σᶻ₁ + -σᶻ₂]
 [(1.0 + 0.0im)]

7-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [σˣ₁ + σˣ₂]
 [σʸ₁ + σʸ₂]
 [σᶻ₁ + σᶻ₂]
 [σˣ₁ + -σˣ₂]
 [σʸ₁ + -σʸ₂]
 [σᶻ₁ + -σᶻ₂]
 [(1.0 + 0.0im)]

Let's try this for 3 sites. First without symmetry.

In [6]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
    symmetry = false,
)
@show bound

Iter: 10 Ap: 9.68e-01 Pobj: -6.0000000e+00 Ad: 9.68e-01 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 3.62e-16 
Relative dual infeasibility: 7.57e-10 
Real Relative Gap: 5.30e-10 
XZ Relative Gap: 1.01e-09 
DIMACS error measures: 5.39e-16 0.00e+00 1.83e-09 0.00e+00 5.30e-10 1.01e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.0903534e+01 Ad: 4.30e-01 Dobj: -4.7695359e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -4.9924435e+01 Ad: 9.57e-01 Dobj: -8.3983133e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.1770206e+01 Ad: 8.94e-01 Dobj: -1.3582785e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -6.4620535e+00 Ad: 8.22e-01 Dobj: -2.9138980e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -4.8114703e+00 Ad: 9.06e-01 Dobj: -4.2755879e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -4.5209992e+00 Ad: 9.47e-01 Dobj: -4.4802497e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.5011955e+00 Ad: 9.65e-0

10-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃

-4.500000018127581

Now with symmetry.

In [7]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
)
@show bound

Iter: 11 Ap: 9.58e-01 Pobj: -4.5000000e+00 Ad: 9.58e-01 Dobj: -4.5000000e+00 
Success: SDP solved
Primal objective value: -4.5000000e+00 
Dual objective value: -4.5000000e+00 
Relative primal infeasibility: 3.26e-16 
Relative dual infeasibility: 1.72e-09 
Real Relative Gap: 1.86e-09 
XZ Relative Gap: 3.24e-09 
DIMACS error measures: 6.52e-16 0.00e+00 4.51e-09 0.00e+00 1.86e-09 3.24e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.43e-01 Pobj: -5.2740518e+00 Ad: 8.34e-01 Dobj: -5.6409413e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.3913022e+01 Ad: 9.25e-01 Dobj: -2.6711480e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -7.3750123e+00 Ad: 9.22e-01 Dobj: -3.5379465e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -4.7955366e+00 Ad: 8.90e-01 Dobj: -4.3281828e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -4.5162512e+00 Ad: 9.32e-01 Dobj: -4.4834263e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -4.5010207e+00 Ad: 9.59e-01 Dobj: -4.4990012e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.5000597e+00 Ad: 9.77e-0

10-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃

-4.500000006400467

Let's look at the isotypical basis.

In [8]:
display([M.basis.polynomials for M in ν.sub_moment_matrices])

10-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [σˣ₁ + σˣ₂ + σˣ₃]
 [σʸ₁ + σʸ₂ + σʸ₃]
 [σᶻ₁ + σᶻ₂ + σᶻ₃]
 [σˣ₁ + (-0.4999999999999998 + 0.8660254037844387im)σˣ₂ + (-0.5000000000000004 - 0.8660254037844385im)σˣ₃]
 [σʸ₁ + (-0.4999999999999998 + 0.8660254037844387im)σʸ₂ + (-0.5000000000000004 - 0.8660254037844385im)σʸ₃]
 [σᶻ₁ + (-0.4999999999999998 + 0.8660254037844387im)σᶻ₂ + (-0.5000000000000004 - 0.8660254037844385im)σᶻ₃]
 [σˣ₁ + (-0.5000000000000004 - 0.8660254037844385im)σˣ₂ + (-0.4999999999999998 + 0.8660254037844387im)σˣ₃]
 [σʸ₁ + (-0.5000000000000004 - 0.8660254037844385im)σʸ₂ + (-0.4999999999999998 + 0.8660254037844387im)σʸ₃]
 [σᶻ₁ + (-0.5000000000000004 - 0.8660254037844385im)σᶻ₂ + (-0.4999999999999998 + 0.8660254037844387im)σᶻ₃]
 [(1.0 + 0.0im)]

Now let's define a function for our common use case.

In [9]:
function f(L, d=1, consecutive=false; symmetry=true)
    @show L
    println("***")
    @show d
    bound, gram, ν = @time hamiltonian_energy(
        L,
        2d,
        solver,
        consecutive=consecutive,
        symmetry=symmetry,
    )
    @show bound
    for M in ν.sub_moment_matrices
        display(M.basis.polynomials)
    end
    println("E/N = ", bound / L)
    println("------------------------------------")
end

f(6, 1, true)

L = 6
***
d = 1
Iter: 10 Ap: 9.68e-01 Pobj: -4.5000000e+00 Ad: 9.68e-01 Dobj: -4.5000000e+00 
Success: SDP solved
Primal objective value: -4.5000000e+00 
Dual objective value: -4.5000000e+00 
Relative primal infeasibility: 6.38e-16 
Relative dual infeasibility: 1.07e-09 
Real Relative Gap: 7.65e-10 
XZ Relative Gap: 1.36e-09 
DIMACS error measures: 1.28e-15 0.00e+00 2.89e-09 0.00e+00 7.65e-10 1.36e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.41e-01 Pobj: -4.0668056e+00 Ad: 8.61e-01 Dobj: -5.2018015e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -5.6056177e+01 Ad: 9.36e-01 Dobj: -5.5398751e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -2.2189805e+01 Ad: 8.38e-01 Dobj: -8.5447620e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -2.1429608e+01 Ad: 7.30e-01 Dobj: -1.4881165e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.8342881e+01 Ad: 8.69e-01 Dobj: -1.7133133e+01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.8022311e+01 Ad: 9.28e-01 Dobj: -1.7891497e+01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.8001586

19-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃
 σˣ₄
 σʸ₄
 σᶻ₄
 σˣ₅
 σʸ₅
 σᶻ₅
 σˣ₆
 σʸ₆
 σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁ + σˣ₂ + σˣ₃ + σˣ₄ + σˣ₅ + σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁ + σʸ₂ + σʸ₃ + σʸ₄ + σʸ₅ + σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σᶻ₁ + σᶻ₂ + σᶻ₃ + σᶻ₄ + σᶻ₅ + σᶻ₆

...

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σᶻ₁ + (0.4999999999999998 - 0.8660254037844387im)σᶻ₂ + (-0.5000000000000004 - 0.8660254037844385im)σᶻ₃ + -σᶻ₄ + (-0.4999999999999998 + 0.8660254037844387im)σᶻ₅ + (0.5000000000000004 + 0.8660254037844385im)σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁ + -σˣ₂ + σˣ₃ + -σˣ₄ + σˣ₅ + -σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁ + -σʸ₂ + σʸ₃ + -σʸ₄ + σʸ₅ + -σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σᶻ₁ + -σᶻ₂ + σᶻ₃ + -σᶻ₄ + σᶻ₅ + -σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (1.0 + 0.0im)

| id     | irep 1 | irep 2 | irep 3 | irep 4 |
|--------|--------|--------|--------|--------|
| degree | 2      | 3      | 1      | 3      |
| mult 2 | 1      | 3      | 2      | 1      |
| mult 3 | 3      | 6      | 4      | 3      |
| mult 4 | 6      | 10     | 7      | 6      |
| mult 5 | 10     | 15     | 11     | 10     |
| mult 6 | 15     | 21     | 16     | 15     |
| mult 7 | 21     | 28     | 22     | 21     |

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*